In [1]:
import pandas as pd
import numpy as np
# warnings
import warnings
import datetime

warnings.filterwarnings('ignore')
df = pd.read_csv('basic_analytics_v1.csv')
# exclude 'cusip_id','trd_exctn_dt','DATED_DATE','vwap_yld' and assign rest of the columns to df1
df1 = df.drop(['cusip_id','trd_exctn_dt','DATED_DATE','vwap_yld'], axis=1)
df = df[['cusip_id','trd_exctn_dt','DATED_DATE','vwap_yld']]
pd.set_option('display.max_columns', None)
# convert stlmnt_dt column using this format 26/09/2018
df['trd_exctn_dt'] = pd.to_datetime(df['trd_exctn_dt'], format='%d/%m/%Y')
df['DATED_DATE'] = pd.to_datetime(df['DATED_DATE'], format='%d/%m/%Y')

In [2]:
# get usa holiday list from librar
from pandas.tseries.holiday import USFederalHolidayCalendar # Calendar for US Federal Holidays
cal = USFederalHolidayCalendar()
# cal will be used in np.busday_count
# holidays is a list of holidays between min and max date in df['trd_exctn_dt']
holidays = cal.holidays(start=df['DATED_DATE'].min(), end=df['trd_exctn_dt'].max())
# convert holidays to a suitable format for np.busday_count
us_holidays = holidays.strftime('%Y-%m-%d').tolist()

In [ ]:
hol = ['2018-03-04','2018-03-05','2018-03-08']

,cusip_id,trd_exctn_dt,DATED_DATE,vwap_yld
0,40055QWT5,2018-09-24,2018-09-28,0.003530
1,40055QWT5,2019-12-18,2018-09-28,0.002988
2,46647MX99,2018-12-12,2017-10-04,0.087648
3,46647MX99,2019-04-03,2017-10-04,0.015075
4,46647MX99,2019-04-09,2017-10-04,0.031332
...,...,...,...,...
126893,989822AA9,2020-01-15,1993-10-20,0.024715
126894,989822AA9,2020-01-22,1993-10-20,0.023428
126895,989822AA9,2020-01-27,1993-10-20,0.024220
126896,989822AA9,2020-01-28,1993-10-20,0.023476


In [11]:
us_holidays[760:]

['2018-02-19',
 '2018-05-28',
 '2018-07-04',
 '2018-09-03',
 '2018-10-08',
 '2018-11-12',
 '2018-11-22',
 '2018-12-25',
 '2019-01-01',
 '2019-01-21',
 '2019-02-18',
 '2019-05-27',
 '2019-07-04',
 '2019-09-02',
 '2019-10-14',
 '2019-11-11',
 '2019-11-28',
 '2019-12-25',
 '2020-01-01',
 '2020-01-20',
 '2018-03-04']

In [284]:
# group by cusip_id and sort by trd_exctn_dt
df = df.groupby('cusip_id').apply(lambda x: x.sort_values(by=['trd_exctn_dt']))
df.reset_index(drop=True, inplace=True)

df['yld_chg_sq'] = df.groupby('cusip_id')['vwap_yld'].diff()**2 # square the difference  
df['trd_exctn_dtholidays'] = df.groupby('cusip_id')['trd_exctn_dt'].shift(1)


In [285]:
# convert DATED_DATE type 
df['DATED_DATE'] = df['DATED_DATE'].astype('datetime64[D]')
# Put data from DATED_DATE_copy if trd_exctn_dtholidays is NaT ----->  Put Data from DATED_DATE for each cusip_ids first row
df['trd_exctn_dtholidays_new'] = np.where(df['trd_exctn_dtholidays'].isna(), df['DATED_DATE'], df['trd_exctn_dtholidays'])

In [286]:
df

,cusip_id,trd_exctn_dt,DATED_DATE,vwap_yld,yld_chg_sq,trd_exctn_dtholidays,trd_exctn_dtholidays_new
0,40055QWT5,2018-09-24,2018-09-28,0.003530,NaN,NaT,2018-09-28
1,40055QWT5,2019-12-18,2018-09-28,0.002988,2.937830e-07,2018-09-24,2018-09-24
2,46647MX99,2018-12-12,2017-10-04,0.087648,NaN,NaT,2017-10-04
3,46647MX99,2019-04-03,2017-10-04,0.015075,5.266894e-03,2018-12-12,2018-12-12
4,46647MX99,2019-04-09,2017-10-04,0.031332,2.642853e-04,2019-04-03,2019-04-03
...,...,...,...,...,...,...,...
126893,989822AA9,2020-01-15,1993-10-20,0.024715,4.934316e-07,2020-01-13,2020-01-13
126894,989822AA9,2020-01-22,1993-10-20,0.023428,1.655089e-06,2020-01-15,2020-01-15
126895,989822AA9,2020-01-27,1993-10-20,0.024220,6.267319e-07,2020-01-22,2020-01-22
126896,989822AA9,2020-01-28,1993-10-20,0.023476,5.524468e-07,2020-01-27,2020-01-27


In [287]:
def LastBusinessDay(row):
    today = row['trd_exctn_dtholidays_new']
    offset = max(1, (today.weekday() + 6) % 7 - 3) 
    timedelta = datetime.timedelta(offset)
    most_recent = today - timedelta
    # whie most_recent is a day from us_holidays, subtract another day 
    while most_recent in us_holidays:
        timedelta = datetime.timedelta(1)
        most_recent = today - timedelta
    return most_recent

In [288]:
# add another index column for using in groupby
df['indexer'] = df.index
# if first row of each cusips trd_exctn_dt is lower than DATED_DATE, then set that rows trd_exctn_dt to DATED_DATE
arr = df.groupby('cusip_id').first()[df.groupby('cusip_id').first()['trd_exctn_dt'] < df.groupby('cusip_id').first()['DATED_DATE']]['indexer'].values
df['trd_exctn_dt'].loc[arr] = df['DATED_DATE'].loc[arr]

# Find business days between trd_exctn_dt and trd_exctn_dtholidays_new ---> column that is created above for calculating business days faster 
df['business_days'] = np.busday_count(df['trd_exctn_dtholidays_new'].values.astype('datetime64[D]'), df['trd_exctn_dt'].values.astype('datetime64[D]'), holidays=us_holidays)

In [289]:
df

,cusip_id,trd_exctn_dt,DATED_DATE,vwap_yld,yld_chg_sq,trd_exctn_dtholidays,trd_exctn_dtholidays_new,indexer,business_days
0,40055QWT5,2018-09-28,2018-09-28,0.003530,NaN,NaT,2018-09-28,0,0
1,40055QWT5,2019-12-18,2018-09-28,0.002988,2.937830e-07,2018-09-24,2018-09-24,1,309
2,46647MX99,2018-12-12,2017-10-04,0.087648,NaN,NaT,2017-10-04,2,297
3,46647MX99,2019-04-03,2017-10-04,0.015075,5.266894e-03,2018-12-12,2018-12-12,3,76
4,46647MX99,2019-04-09,2017-10-04,0.031332,2.642853e-04,2019-04-03,2019-04-03,4,4
...,...,...,...,...,...,...,...,...,...
126893,989822AA9,2020-01-15,1993-10-20,0.024715,4.934316e-07,2020-01-13,2020-01-13,126893,2
126894,989822AA9,2020-01-22,1993-10-20,0.023428,1.655089e-06,2020-01-15,2020-01-15,126894,4
126895,989822AA9,2020-01-27,1993-10-20,0.024220,6.267319e-07,2020-01-22,2020-01-22,126895,3
126896,989822AA9,2020-01-28,1993-10-20,0.023476,5.524468e-07,2020-01-27,2020-01-27,126896,1


In [290]:
arrr = df[df['business_days']  == 0].index
df[df['business_days']  == 0]

,cusip_id,trd_exctn_dt,DATED_DATE,vwap_yld,yld_chg_sq,trd_exctn_dtholidays,trd_exctn_dtholidays_new,indexer,business_days
0,40055QWT5,2018-09-28,2018-09-28,0.003530,NaN,NaT,2018-09-28,0,0
1053,013104AC8,2018-10-09,1996-06-17,0.103428,3.612174e-05,2018-10-08,2018-10-08,1053,0
2642,02005NAV2,2018-11-13,2014-09-29,0.048238,4.621151e-07,2018-11-12,2018-11-12,2642,0
3174,02005NAY6,2018-11-13,2015-02-13,0.044083,1.094080e-06,2018-11-12,2018-11-12,3174,0
3573,02005NAZ3,2019-09-03,2015-03-30,0.026056,3.437797e-05,2019-09-02,2019-09-02,3573,0
...,...,...,...,...,...,...,...,...,...
124379,95001HC41,2019-10-31,2019-10-31,0.002477,NaN,NaT,2019-10-31,124379,0
125662,97650WAG3,2019-06-06,2019-06-06,0.048355,NaN,NaT,2019-06-06,125662,0
126326,984121CL5,2018-10-09,2015-03-03,0.063929,4.207893e-06,2018-10-08,2018-10-08,126326,0
126479,984121CL5,2019-11-12,2015-03-03,0.060836,5.831638e-06,2019-11-11,2019-11-11,126479,0


In [291]:
newdates = df[(df['business_days']  == 0)].apply(lambda x: LastBusinessDay(x), axis=1)
df['trd_exctn_dtholidays_new'].loc[(df['business_days']  == 0)] = newdates
df.loc[arrr]

,cusip_id,trd_exctn_dt,DATED_DATE,vwap_yld,yld_chg_sq,trd_exctn_dtholidays,trd_exctn_dtholidays_new,indexer,business_days
0,40055QWT5,2018-09-28,2018-09-28,0.003530,NaN,NaT,2018-09-27,0,0
1053,013104AC8,2018-10-09,1996-06-17,0.103428,3.612174e-05,2018-10-08,2018-10-05,1053,0
2642,02005NAV2,2018-11-13,2014-09-29,0.048238,4.621151e-07,2018-11-12,2018-11-09,2642,0
3174,02005NAY6,2018-11-13,2015-02-13,0.044083,1.094080e-06,2018-11-12,2018-11-09,3174,0
3573,02005NAZ3,2019-09-03,2015-03-30,0.026056,3.437797e-05,2019-09-02,2019-08-30,3573,0
...,...,...,...,...,...,...,...,...,...
124379,95001HC41,2019-10-31,2019-10-31,0.002477,NaN,NaT,2019-10-30,124379,0
125662,97650WAG3,2019-06-06,2019-06-06,0.048355,NaN,NaT,2019-06-05,125662,0
126326,984121CL5,2018-10-09,2015-03-03,0.063929,4.207893e-06,2018-10-08,2018-10-05,126326,0
126479,984121CL5,2019-11-12,2015-03-03,0.060836,5.831638e-06,2019-11-11,2019-11-08,126479,0


In [292]:
# Calculate Again with replaced last business days for trd_exctn_dtholidays_new
df['business_days'] = np.busday_count(df['trd_exctn_dtholidays_new'].values.astype('datetime64[D]'), df['trd_exctn_dt'].values.astype('datetime64[D]'), holidays=us_holidays)
df.loc[arrr] 

,cusip_id,trd_exctn_dt,DATED_DATE,vwap_yld,yld_chg_sq,trd_exctn_dtholidays,trd_exctn_dtholidays_new,indexer,business_days
0,40055QWT5,2018-09-28,2018-09-28,0.003530,NaN,NaT,2018-09-27,0,1
1053,013104AC8,2018-10-09,1996-06-17,0.103428,3.612174e-05,2018-10-08,2018-10-05,1053,1
2642,02005NAV2,2018-11-13,2014-09-29,0.048238,4.621151e-07,2018-11-12,2018-11-09,2642,1
3174,02005NAY6,2018-11-13,2015-02-13,0.044083,1.094080e-06,2018-11-12,2018-11-09,3174,1
3573,02005NAZ3,2019-09-03,2015-03-30,0.026056,3.437797e-05,2019-09-02,2019-08-30,3573,1
...,...,...,...,...,...,...,...,...,...
124379,95001HC41,2019-10-31,2019-10-31,0.002477,NaN,NaT,2019-10-30,124379,1
125662,97650WAG3,2019-06-06,2019-06-06,0.048355,NaN,NaT,2019-06-05,125662,1
126326,984121CL5,2018-10-09,2015-03-03,0.063929,4.207893e-06,2018-10-08,2018-10-05,126326,1
126479,984121CL5,2019-11-12,2015-03-03,0.060836,5.831638e-06,2019-11-11,2019-11-08,126479,1


In [293]:
# convert trd_exctn_dt_diff to years
df['time_diff_yrs'] = df['business_days']/252
df


,cusip_id,trd_exctn_dt,DATED_DATE,vwap_yld,yld_chg_sq,trd_exctn_dtholidays,trd_exctn_dtholidays_new,indexer,business_days,time_diff_yrs
0,40055QWT5,2018-09-28,2018-09-28,0.003530,NaN,NaT,2018-09-27,0,1,0.003968
1,40055QWT5,2019-12-18,2018-09-28,0.002988,2.937830e-07,2018-09-24,2018-09-24,1,309,1.226190
2,46647MX99,2018-12-12,2017-10-04,0.087648,NaN,NaT,2017-10-04,2,297,1.178571
3,46647MX99,2019-04-03,2017-10-04,0.015075,5.266894e-03,2018-12-12,2018-12-12,3,76,0.301587
4,46647MX99,2019-04-09,2017-10-04,0.031332,2.642853e-04,2019-04-03,2019-04-03,4,4,0.015873
...,...,...,...,...,...,...,...,...,...,...
126893,989822AA9,2020-01-15,1993-10-20,0.024715,4.934316e-07,2020-01-13,2020-01-13,126893,2,0.007937
126894,989822AA9,2020-01-22,1993-10-20,0.023428,1.655089e-06,2020-01-15,2020-01-15,126894,4,0.015873
126895,989822AA9,2020-01-27,1993-10-20,0.024220,6.267319e-07,2020-01-22,2020-01-22,126895,3,0.011905
126896,989822AA9,2020-01-28,1993-10-20,0.023476,5.524468e-07,2020-01-27,2020-01-27,126896,1,0.003968


In [294]:
# yld_chg_sq divided by time_diff_yrs
df['yld_chg_per_time'] = df['yld_chg_sq']/df['time_diff_yrs']
df

,cusip_id,trd_exctn_dt,DATED_DATE,vwap_yld,yld_chg_sq,trd_exctn_dtholidays,trd_exctn_dtholidays_new,indexer,business_days,time_diff_yrs,yld_chg_per_time
0,40055QWT5,2018-09-28,2018-09-28,0.003530,NaN,NaT,2018-09-27,0,1,0.003968,NaN
1,40055QWT5,2019-12-18,2018-09-28,0.002988,2.937830e-07,2018-09-24,2018-09-24,1,309,1.226190,2.395900e-07
2,46647MX99,2018-12-12,2017-10-04,0.087648,NaN,NaT,2017-10-04,2,297,1.178571,NaN
3,46647MX99,2019-04-03,2017-10-04,0.015075,5.266894e-03,2018-12-12,2018-12-12,3,76,0.301587,1.746391e-02
4,46647MX99,2019-04-09,2017-10-04,0.031332,2.642853e-04,2019-04-03,2019-04-03,4,4,0.015873,1.664997e-02
...,...,...,...,...,...,...,...,...,...,...,...
126893,989822AA9,2020-01-15,1993-10-20,0.024715,4.934316e-07,2020-01-13,2020-01-13,126893,2,0.007937,6.217238e-05
126894,989822AA9,2020-01-22,1993-10-20,0.023428,1.655089e-06,2020-01-15,2020-01-15,126894,4,0.015873,1.042706e-04
126895,989822AA9,2020-01-27,1993-10-20,0.024220,6.267319e-07,2020-01-22,2020-01-22,126895,3,0.011905,5.264548e-05
126896,989822AA9,2020-01-28,1993-10-20,0.023476,5.524468e-07,2020-01-27,2020-01-27,126896,1,0.003968,1.392166e-04


In [295]:
# Find days between trd_exctn_dtholidays_new trd_exctn_dt and assign into a column called NormalDays
df['NormalDays'] = (df['trd_exctn_dt'] - df['trd_exctn_dtholidays_new']).dt.days

# groupby cusip_id and find cumulative sum of trd_exctn_dt_diff
df['trd_exctn_dt_diff_cumsum'] = df.groupby('cusip_id')['NormalDays'].cumsum()  
# drop indexer
df.drop('indexer', axis=1, inplace=True)

In [310]:
# Volatility Function
def Volatility(df, days, N=0):    
    df['Step5'] = 0
    def Step5new(cusipdf, days, N ):
        steplist = [] 
        for i in range(cusipdf.shape[0]): 
            trd_exctn_dt_diff_cumsum = cusipdf.trd_exctn_dt_diff_cumsum.values
            yld_chg_per_time = cusipdf.yld_chg_per_time.values
            last30 = trd_exctn_dt_diff_cumsum[i] - days            
                 
            if  (last30 < 0) or( last30 > trd_exctn_dt_diff_cumsum[i] )or (i+1 <= N):
                steplist.append(-999)
            elif i+1 > N and last30 <= trd_exctn_dt_diff_cumsum[i]:
                # get loc with :i for getting all rows before i            
                steplist.append(yld_chg_per_time[1:i+1].sum())
                
        if len(steplist) != cusipdf.shape[0]:
            print('error')
            
        df['Step5'].loc[cusipdf.index] = steplist
       
                
    print("Calculating yld_sq_",days,"_days")
    df.groupby('cusip_id').apply(lambda x: Step5new(x, days=days, N=N ))           
    df['yld_sq_'+str(days)+'_days'] = df['Step5']   
    df['yld_variance'] = df['yld_sq_'+str(days)+'_days'] * (1/(df.groupby('cusip_id').cumcount()))
    df['yld_volatility'+str(days)+'_days'] = df['yld_variance'].apply(lambda x: np.sqrt(x))    
    try:
        df.drop(["yld_variance","yld_sq_"+str(days)+"_days","Step5","NormalDays","time_diff_yrs","business_days","trd_exctn_dtholidays_new","trd_exctn_dtholidays","yld_chg_sq"],axis=1,inplace=True)
    except:
        # drop
        df.drop(["yld_sq_"+str(days)+"_days","yld_variance","Step5"],axis=1,inplace=True)
    return df

In [313]:
# use 
df = Volatility(df, 360, 1)

Calculating yld_sq_ 360 _days


In [315]:
df.to_csv('output.csv', index=False)

In [316]:
dfresult = pd.concat([df1,df],axis=1)

In [317]:
dfresult

,Var1,rptd_pr,entrd_vol_qt,stlmnt_dt,yld_pt,ISSUE_ID,ISSUER_ID,ISSUER_CUSIP,COMPLETE_CUSIP,CUSIP_NAME,COUNTRY_DOMICILE,OFFERING_AMT,AMOUNT_OUTSTANDING,OFFERING_YIELD,OFFERING_AMT_1,OFFERING_PRICE,PREFERRED_SECURITY,PRINCIPAL_AMT,COUPON,FIRST_INTEREST_DATE,LAST_INTEREST_DATE,INTEREST_FREQUENCY,MATURITY,INDUSTRY_GROUP,INDUSTRY_CODE,PARENT_ID,NAICS_CODE,SIC_CODE,COUNTRY,vwap,daily_volume,daily_trades,ModDuration,YearDuration,PerDuration,YearConvexity,PerConvexity,AGE,cusip_id,trd_exctn_dt,DATED_DATE,vwap_yld,yld_chg_per_time,trd_exctn_dt_diff_cumsum,yld_volatility90_days,yld_volatility30_days,yld_volatility360_days
0,67383,100.000,25000.0,26/09/2018,NaN,777809,40163,NaN,40055QWT5,GS FIN CORP,USA,264.0,264.0,NaN,264.0,100.000,N,1000,0.000,NaT,NaT,0,28/09/2023,2,22,1849,52311.0,6211.0,USA,98.250000,280000.0,12,4.996615,5.005435,10.010870,27.460068,109.840271,-0.005556,40055QWT5,2018-09-28,2018-09-28,0.003530,NaN,1,NaN,NaN,NaN
1,67384,98.880,5000.0,20/12/2019,NaN,777809,40163,NaN,40055QWT5,GS FIN CORP,USA,264.0,264.0,NaN,264.0,100.000,N,1000,0.000,NaT,NaT,0,28/09/2023,2,22,1849,52311.0,6211.0,USA,98.880000,15000.0,3,3.766351,3.771978,7.543956,16.065763,64.263051,1.227778,40055QWT5,2019-12-18,2018-09-28,0.002988,2.395900e-07,451,0.000489,0.000489,0.000489
2,78705,93.350,6000.0,14/12/2018,NaN,709305,47149,NaN,46647MX99,JPMORGAN CHASE & CO,USA,1792.0,0.0,NaN,1792.0,100.000,N,1000,0.000,NaT,NaT,0,03/10/2019,2,20,69841,NaN,6021.0,USA,93.350000,18000.0,3,0.768518,0.802198,1.604396,0.958746,3.834985,1.194444,46647MX99,2018-12-12,2017-10-04,0.087648,NaN,434,NaN,NaN,NaN
3,78706,99.260,30000.0,05/04/2019,NaN,709305,47149,NaN,46647MX99,JPMORGAN CHASE & CO,USA,1792.0,0.0,NaN,1792.0,100.000,N,1000,0.000,NaT,NaT,0,03/10/2019,2,20,69841,NaN,6021.0,USA,99.260000,90000.0,3,0.490836,0.494536,0.989071,0.484501,1.938006,1.502778,46647MX99,2019-04-03,2017-10-04,0.015075,1.746391e-02,546,0.132151,0.132151,0.132151
4,78707,98.320,10000.0,11/04/2019,NaN,709305,47149,NaN,46647MX99,JPMORGAN CHASE & CO,USA,1792.0,0.0,NaN,1792.0,100.000,N,1000,0.000,NaT,NaT,0,03/10/2019,2,20,69841,NaN,6021.0,USA,98.524000,50000.0,5,0.470765,0.478142,0.956284,0.453370,1.813479,1.519444,46647MX99,2019-04-09,2017-10-04,0.031332,1.664997e-02,552,0.130602,0.130602,0.130602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126893,126893,116.938,8000.0,17/01/2020,2.369647,24074,4613,989822.0,989822AA9,ZURICH REINSURANCE CENTRE HLDGS INC,USA,200000.0,200000.0,7.192,200000.0,99.178,N,1000,7.125,15/04/1994,15/04/2023,2,15/10/2023,2,23,72799,524126.0,6331.0,USA,116.543832,165000.0,18,3.293681,3.334382,6.668765,13.316497,53.265989,26.241667,989822AA9,2020-01-15,1993-10-20,0.024715,6.217238e-05,9583,0.019243,0.019243,0.019243
126894,126894,117.224,20000.0,24/01/2020,2.274800,24074,4613,989822.0,989822AA9,ZURICH REINSURANCE CENTRE HLDGS INC,USA,200000.0,200000.0,7.192,200000.0,99.178,N,1000,7.125,15/04/1994,15/04/2023,2,15/10/2023,2,23,72799,524126.0,6331.0,USA,116.961111,90000.0,6,3.277953,3.316352,6.632703,13.205275,52.821099,26.261111,989822AA9,2020-01-22,1993-10-20,0.023428,1.042706e-04,9590,0.019206,0.019206,0.019206
126895,126895,116.590,250000.0,29/01/2020,2.422226,24074,4613,989822.0,989822AA9,ZURICH REINSURANCE CENTRE HLDGS INC,USA,200000.0,200000.0,7.192,200000.0,99.178,N,1000,7.125,15/04/1994,15/04/2023,2,15/10/2023,2,23,72799,524126.0,6331.0,USA,116.593057,1259000.0,8,3.262508,3.302017,6.604034,13.096685,52.386741,26.275000,989822AA9,2020-01-27,1993-10-20,0.024220,5.264548e-05,9595,0.019162,0.019162,0.019162
126896,126896,117.528,10000.0,30/01/2020,2.177909,24074,4613,989822.0,989822AA9,ZURICH REINSURANCE CENTRE HLDGS INC,USA,200000.0,200000.0,7.192,200000.0,99.178,N,1000,7.125,15/04/1994,15/04/2023,2,15/10/2023,2,23,72799,524126.0,6331.0,USA,116.869000,60000.0,4,3.261631,3.299917,6.599834,13.090471,52.361883,26.277778,989822AA9,2020-01-28

In [ ]:
# Codes for finding what was causing the 0's in the business day column 


# df[(df['business_days']  == 0) & (df['trd_exctn_dtholidays_new'].isin(us_holidays) == False)]
# if trd_exctn_dtholidays_new is 1 day before and trd_exctn_dtholidays_new is a holiday show rows
# df[(df['trd_exctn_dtholidays_new'] == df['trd_exctn_dt'] - pd.Timedelta(days=1) ) & (df['trd_exctn_dtholidays_new'].isin(us_holidays))]
# pd set opt show max row 140
# pd.set_option('display.max_rows', 140)
# zf[['cusip_id','trd_exctn_dt','trd_exctn_dtholidays_new','business_days','time_diff_yrs','yld_chg_sq','yld_chg_per_time']].loc[92150:]